<a href="https://colab.research.google.com/github/juliapimentelma/desafio-unifacisa/blob/main/Quest%C3%A3o_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dataset de doenças cardiacas
# data 15/10/2025
# disciplina desafio unifacisa - dados e ia
# autor: Júlia Pimentel

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.svm import SVC

# Importando o Keras e TensorFlow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [ ]:
# Create a DataFrame from the housing dataset
df = pd.DataFrame(housing.data, columns=housing.feature_names)
TARGET = 'MedHouseVal'  # Preço Mediano do Imóvel (exemplo do dataset da Califórnia)
df[TARGET] = housing.target

display(df.head())

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
TARGET = 'MedHouseVal'  # Preço Mediano do Imóvel (exemplo do dataset da Califórnia)

# Create new features
df['RoomsPerPerson'] = df['AveRooms'] / df['AveOccup']
df['IncomePerPerson'] = df['MedInc'] / df['AveOccup']
df['AveOtherRooms'] = df['AveRooms'] - df['AveBedrms']

# As features (atributos originais + os criados na Feature Engineering)
FEATURES = [
    'MedInc',
    'HouseAge',
    'AveRooms',
    'AveBedrms',
    'AveOccup',
    'RoomsPerPerson',   # Feature criada
    'IncomePerPerson',  # Feature criada
    'AveOtherRooms'     # Feature criada
]

X = df[FEATURES]
y = df[TARGET]

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Escalonamento dos Dados (coloca todos os dados na mesma escala)
scaler = StandardScaler()

# Ajusta o scaler SOMENTE nos dados de TREINO e transforma
X_train_scaled = scaler.fit_transform(X_train)

# Aplica a mesma transformação nos dados de TESTE
X_test_scaled = scaler.transform(X_test)

# Obtém o número de features para definir a camada de entrada
input_shape = X_train_scaled.shape[1]

In [ ]:
# Treinamento do Modelo de Regressão Linear

# Inicializa o modelo
linear_model = LinearRegression()

# Treina o modelo usando os dados escalonados de treino
linear_model.fit(X_train_scaled, y_train)

LinearRegression()

In [ ]:
# Inicializa o modelo XGBoost para regressão (Modelo Básico)
xgb_basic = XGBRegressor(random_state=42,
                         n_estimators=100, # Número inicial de árvores
                         learning_rate=0.1,
                         objective='reg:squarederror') # Objetivo para regressão

# Treina o modelo básico
xgb_basic.fit(X_train, y_train)

# Faz previsões (para comparação)
y_pred_basic = xgb_basic.predict(X_test)
rmse_basic = np.sqrt(mean_squared_error(y_test, y_pred_basic))
r2_basic = r2_score(y_test, y_pred_basic)

print("--- Resultado do XGBoost (Básico) ---")
print(f"RMSE Básico: {rmse_basic:.4f}")
print(f"R² Básico: {r2_basic:.4f}")
print("-" * 40)



# Otimização de Hiperparâmetros com GridSearchCV (máximo de performance do XGBoost)
param_grid = {
    'n_estimators': [200, 500],   # Número de árvores
    'max_depth': [3, 5],    # Profundidade máxima da árvore
    'learning_rate': [0.05, 0.1],   # Taxa de aprendizado
    'subsample': [0.7, 1.0]   # Fração de observações para amostragem
}

# Inicializa o modelo para a busca em grade
xgb_grid = XGBRegressor(random_state=42,
                        objective='reg:squarederror',
                        use_label_encoder=False,
                        eval_metric='rmse')

# Configura a busca em GridSearchCV e busca minimizar o erro quadrático negativo
grid_search = GridSearchCV(estimator=xgb_grid,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=3,
                           verbose=1,
                           n_jobs=-1) # Usa todos os núcleos da CPU

# Executa a busca
print("Iniciando Busca em Grade (Grid Search)...")
grid_search.fit(X_train, y_train)

# Pega os melhores parâmetros
best_params = grid_search.best_params_
print("\nMelhores Parâmetros Encontrados:")
print(best_params)

--- Resultado do XGBoost (Básico) ---
RMSE Básico: 0.6516
R² Básico: 0.6760
----------------------------------------
Iniciando Busca em Grade (Grid Search)...
Fitting 3 folds for each of 16 candidates, totalling 48 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [23:31:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Melhores Parâmetros Encontrados:
{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.7}


In [ ]:
# Definição da Arquitetura da Rede Neural
def build_ann_model(input_shape):
    model = Sequential()

    model.add(Dense(64, activation='relu', input_shape=(input_shape,))) # Camada de Entrada e Primeira Camada Oculta
    model.add(Dense(32, activation='relu')) # Segunda Camada Oculta
    model.add(Dense(1, activation='linear')) # Camada de Saída

    return model

ann_model = build_ann_model(input_shape)

# Usamos o otimizador Adam e a função de perda MSE para regressão
ann_model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse', # MSE é o padrão para regressão
                  metrics=['mae']) # Monitoramos MAE (Erro Absoluto Médio)

# Configurações para Treinamento (Callbacks)
# EarlyStopping evita o overfitting, parando o treino quando o erro de validação para de diminuir
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

# 4. Treinamento do Modelo
print("Iniciando Treinamento da Rede Neural...")

# O treinamento utiliza uma porção dos dados de treino como validação (validation_split)
history = ann_model.fit(
    X_train_scaled,
    y_train,
    epochs=100, # Número máximo de épocas (o EarlyStopping irá pará-lo mais cedo)
    batch_size=32,
    validation_split=0.1, # 10% dos dados de treino serão usados para validação
    callbacks=[early_stop],
    verbose=1
)



Iniciando Treinamento da Rede Neural...
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.6353 - mae: 0.8793 - val_loss: 0.4860 - val_mae: 0.5198
Epoch 2/100
465/465 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4539 - mae: 0.4864 - val_loss: 0.4650 - val_mae: 0.4996
Epoch 3/100
465/465 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4559 - mae: 0.4827 - val_loss: 0.4584 - val_mae: 0.4971
Epoch 4/100
465/465 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4589 - mae: 0.4732 - val_loss: 0.4518 - val_mae: 0.4869
Epoch 5/100
465/465 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4194 - mae: 0.4671 - val_loss: 0.4507 - val_mae: 0.4862
Epoch 6/100
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4180 - mae: 0.4630 - val_loss: 0.4461 - val_mae: 0.4777
Epoch 7/100
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4098 - mae: 0.4592 - val_loss: 0.4392 - val_mae: 0.4774
Epoch 8/100
465/465 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4070 - mae: 0.4579 - val_loss: 0.4395 - val_mae: 0.4789
Epoch 9/100
465/465 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss:

In [ ]:
# VALIANDO os modelos com métricas como RMSE e R²

# Avaliação do Modelo de REGRESSÃO LINEAR
y_pred = linear_model.predict(X_test_scaled)

# Cálculo das métricas de avaliação
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("--- Resultado da Regressão Linear ---")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse:.4f}")
print(f"Erro Absoluto Médio (MAE): {mae:.4f}")
print(f"Coeficiente de Determinação (R²): {r2:.4f}")


# Avaliação do Modelo Otimizado XGBoost
xgb_optimized = grid_search.best_estimator_
y_pred_optimized = xgb_optimized.predict(X_test)

# Cálculo das métricas do modelo otimizado
rmse_optimized = np.sqrt(mean_squared_error(y_test, y_pred_optimized))
mae_optimized = mean_absolute_error(y_test, y_pred_optimized)
r2_optimized = r2_score(y_test, y_pred_optimized)

print("\n--- Resultado do XGBoost (Otimizado) ---")
print(f"RMSE Otimizado: {rmse_optimized:.4f}")
print(f"MAE Otimizado: {mae_optimized:.4f}")
print(f"R² Otimizado: {r2_optimized:.4f}")

# 4. Análise de Importância das Features
feature_importance = pd.Series(xgb_optimized.feature_importances_, index=FEATURES).sort_values(ascending=False)

print("\n--- Importância das Features no XGBoost ---")
print(feature_importance)


# Avaliação do Modelo REDES NEURAIS ARTIFICIAIS
print("\nAvaliando o Modelo no Conjunto de Teste...")
# Avalia a perda (MSE) e o MAE no conjunto de teste
loss, mae_test = ann_model.evaluate(X_test_scaled, y_test, verbose=0)

# Fazendo as previsões para calcular o RMSE e R²
y_pred_ann = ann_model.predict(X_test_scaled).flatten()

rmse_ann = np.sqrt(loss) # A raiz quadrada da perda (MSE) é o RMSE
r2_ann = r2_score(y_test, y_pred_ann)
mae_ann = mae_test

print("\n--- Resultado da Rede Neural Artificial ---")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse_ann:.4f}")
print(f"Erro Absoluto Médio (MAE): {mae_ann:.4f}")
print(f"Coeficiente de Determinação (R²): {r2_ann:.4f}")

--- Resultado da Regressão Linear ---
Raiz do Erro Quadrático Médio (RMSE): 0.7402
Erro Absoluto Médio (MAE): 0.5374
Coeficiente de Determinação (R²): 0.5819

--- Resultado do XGBoost (Otimizado) ---
RMSE Otimizado: 0.6500
MAE Otimizado: 0.4616
R² Otimizado: 0.6776

--- Importância das Features no XGBoost ---
IncomePerPerson    0.756434
AveOtherRooms      0.055852
HouseAge           0.050982
MedInc             0.043112
RoomsPerPerson     0.028252
AveBedrms          0.025162
AveOccup           0.022327
AveRooms           0.017880
dtype: float32

Avaliando o Modelo no Conjunto de Teste...
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

--- Resultado da Rede Neural Artificial ---
Raiz do Erro Quadrático Médio (RMSE): 0.6430
Erro Absoluto Médio (MAE): 0.4583
Coeficiente de Determinação (R²): 0.6845
